In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputers

In [4]:
df = pd.read_csv("./data/Training_Data_0611N_cleaned.csv")
df.head()
df.shape

,cust_id,profitable_flag,count_accts,cm_age,flag_top_ed_spender,flag_cust_fee_paid_6m,pre6m_cust_spend,pre6m_cust_roc_cnt,pre6m_cust_non_disc_amt,pre6m_cust_non_disc_cnt,...,cnsumr_chrg_actv_cust_cnt,cnsumr_chrg_avg_credit_12m_amt,cnsumr_lend_actv_cust_cnt,cnsumr_lend_tot_util_ratio,sow_revol_avg_paydown_pct,sow_tot_annual_ext_pmt_amt,sow_tot_revol_bal_amt,sow_tot_revol_cnt,sow_tot_trans_bal_amt,sow_tot_trans_cnt
0,550391.0,0.0,1.0,74.0,1.0,0.0,1518.0,122.0,811.0,96.0,...,0.0,NaN,1.0,0.034044,0.0000,991.0,NaN,0.0,NaN,3.0
1,550392.0,0.0,1.0,64.0,0.0,0.0,4807.0,354.0,1052.0,82.0,...,0.0,7.0,1.0,0.044086,0.0877,2.0,24.0,2.0,NaN,0.0
2,550393.0,0.0,1.0,75.0,1.0,0.0,2361.0,204.0,1351.0,118.0,...,0.0,NaN,1.0,0.039613,0.0000,4828.0,NaN,0.0,387.0,3.0
3,550394.0,0.0,2.0,73.0,0.0,1.0,5452.0,142.0,1195.0,50.0,...,1.0,493.0,1.0,0.001200,0.0000,4598.0,NaN,0.0,295.0,4.0
4,550395.0,0.0,2.0,68.0,0.0,0.0,170.0,34.0,88.0,20.0,...,0.0,NaN,2.0,0.003378,0.0168,2176.0,2606.0,5.0,143.0,13.0


(19595, 50)

In [5]:
df.isna().sum()

cust_id                               0
profitable_flag                       0
count_accts                           0
cm_age                                0
flag_top_ed_spender                  48
flag_cust_fee_paid_6m                 0
pre6m_cust_spend                    590
pre6m_cust_roc_cnt                    0
pre6m_cust_non_disc_amt            1182
pre6m_cust_non_disc_cnt               0
pre6m_cust_disc_amt                 693
pre6m_cust_disc_cnt                   0
pre6m_cust_outbound_amt            2177
pre6m_cust_online_amt              1211
pre6m_cust_online_cnt                 0
pre6m_cust_travel_amt              1921
pre6m_cust_travel_cnt                 0
pre6m_cust_retail_amt               834
pre6m_cust_retail_cnt                 0
pre6m_cust_myca_active                0
pre6m_cust_mob_logins                 0
pre6m_total_mc_trs                    0
acq_sub_chn                         902
acq_type_grp                        872
fee_type_grp                          0


### Mean Imputation (removing Categorical Variables)

In [ ]:
df_non_cat = df.select_dtypes(exclude=['object'])

X = df_non_cat.drop(columns='profitable_flag').values
Y = df_non_cat['profitable_flag'].values

X.shape
Y.shape

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
lda = LinearDiscriminantAnalysis()
pipeline = Pipeline(steps=[('imputer', imputer),('model', lda)])
kfold = KFold(n_splits=3, shuffle=True, random_state=1)
result = cross_val_score(pipeline, X, Y, cv=kfold, scoring='accuracy')

print(f"Cross-validation accuracy: {result.mean()}")

### Mean Imputation (removing Categorical Variables & correlated features)

In [24]:
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
threshold = 0.05
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]


df_dropped = df.drop(columns=to_drop)
df_dropped = df_dropped.select_dtypes(exclude=['object'])

X = df_dropped.drop(columns='profitable_flag').values
Y = df_dropped['profitable_flag'].values

X.shape
Y.shape

(19595, 2)

(19595,)

In [27]:
df_dropped.columns

Index(['cust_id', 'profitable_flag', 'count_accts'], dtype='object')

In [23]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
lda = LinearDiscriminantAnalysis()
pipeline = Pipeline(steps=[('imputer', imputer),('model', lda)])
kfold = KFold(n_splits=3, shuffle=True, random_state=1)
result = cross_val_score(pipeline, X, Y, cv=kfold, scoring='accuracy')

print(f"Cross-validation accuracy: {result.mean()}")

Cross-validation accuracy: 0.8140343138408562


### Mean Imputation (including Categorical Variables)

In [ ]:
class DataFrameImputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


In [ ]:
df_t = DataFrameImputer().fit_transform(df)

df_t.isna().any().any()
df_t.shape

In [ ]:
X = df_t.drop('profitable_flag', axis='columns').values
Y = df_t['profitable_flag'].values



lda = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, shuffle=True, random_state=1)

result = cross_val_score(lda, X, Y, cv=kfold, scoring='accuracy')
result